# 便利ツールの紹介

ここでは言語処理を行う上で欠かせないツールについて紹介します。　　

### 目次
- MeCab
- Pandas

## MeCab

### 文章の分かち書き

> 文章を単語の集合に変換  
> 分割された各単語には品詞などの情報が付与

どんな機械学習でも統計学的な手法でも結局は単語単位に処理を行うのでアジア圏の言語ではまず分割するところから入ります。  
ちなみに英語のように単語間がスペースで区切られている言語はスペース毎に分割するだけなのでスーパーイージーです。（その代わり品詞が曖昧過ぎるといった課題もあります。）  

このツールで目指すのは、  
`"文章"　⇒　["単語","単語",...,"単語"]`  
のように最終的には単語のリスト（文字列のリスト）に変換します。    

### 分析の前処理に
> 多くの分析ツール、機械学習のモデルでは文章を単語のリストとしたものを入力とする  

### データ分析
> ただ分割するだけではなく、品詞などもわかる  
> 品詞の分布などで考察もできる


### 使うには
> 作成者がコマンドツールとして提供  
> コマンドを各プログラミング言語で実行し、結果をまた言語内で読み込める形式にしたライブラリが存在
> PythonではMeCab(mecab-python)が存在


1.   mecabのインストール（パッケージ管理やgitのソースから）
2.   各言語でmecabを使うライブラリのインストール、または自作
3.   必要なら目的の処理に適した辞書のインストール

元ソース : https://taku910.github.io/mecab/

In [ ]:
# @title 下準備（ubuntu環境なのでaptでインストールしてます）
# 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null # mecabの利用に必要なライブラリのインストール
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null                    # gitから辞書ファイルのクローン
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1                   # クローンした辞書のインストール
!pip install mecab-python3==0.7 > /dev/null                                                             # 0.7意外だと謎のエラーが発生して安定しない

# シンボリックリンクによるエラー回避
!ln -s /etc/mecabrc /usr/local/etc/mecabrc                                                              # 辞書の参照先にインストール先のディレクトリを追加
!echo `mecab-config --dicdir` "/mecab-ipadic-neologd"                                                    # mecabの設定ファイルに新しく辞書を追加したことを追記

In [2]:
"""
MeCabを使用するオブジェクトの初期化
MeCab.Taggerにオプション（コマンドの解析文章以外の引数）
"""
import MeCab
m = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')                         # mecabはコマンドでは -d の後に追加した辞書のPATHを指定して追加の辞書を扱う
                                                                                                        # MeCab.Taggerには主にコマンドのオプションを文字列で渡す。（なんもない場合はMeCab.Tagger()でもいい）

In [ ]:
"""
基本的にparseToNodeで単語を分割し,返り値のnodeに一個目の単語が入ります。  
二個目以降はnode.nextで参照します。
データが連結リストになっています。
また各nodeは様々なメンバ変数を持っています。例）surfaceは見出し語
また、featuresで,区切りの文字列として解析結果を表示できます。
,で区切ったフィールドにどんな内容があるのかは
https://taku910.github.io/mecab/#usage-toolsで見てみてください
"""
node = m.parseToNode("そうして自身よりも遥かな高みにいるコムギとの対局を続けていくうちに自らの価値観に変化が生じ始め戸惑いを抱く。")

print("nodeの型：",type(node),"\n")

while node:                                                                                             # 条件で、nodeがNoneでないときに表示するのでwhileで回す方が楽

    #品詞闇第五などの情報を取得
    pos = node.feature.split(",")                                                                       # 品詞や見出し語などの情報を文字列のリストに変換
    print("\n単語：",pos[6],"\n品詞：",pos[0])
    #次の単語に進める
    node = node.next

In [ ]:
"""
学習データ生成の例
文章を単語のリストに変換しています。
"""

sample_corpas = [\
                 "家に可愛い人形とか置いてそう",\
                 "発表してる時に理解しやすい言語",\
                 "先行研究で消費電力減らすために",\
                 "今回金井くんの添削大変そうな問題"]
print("【生の学習データの例】:\n\n",sample_corpas)

import MeCab

m = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati')                # -Owakatiというオプションで分かち書き(区切りはスペース)した文書を出力 

output_corpus = ""                                                                                      # 最終的に出力される学習データ
for sentence in sample_corpas:
  output_corpus += m.parse(sentence).split()
print("\n【分かち書き後の学習データ】:\n\n",output_corpus)

## [Pandas](https://pandas.pydata.org/)

pythonのライブラリで表形式のデータを処理するのに非常に便利です。  
この３年生向けのイントロでは触れませんが、データからより学習するために不必要な記号を削除したり改行文字を取り払うなどの前処理をするときなどにも役に立ちます。  
表のデータを列ごとに分析したり、データベースのように検索することもできます。  
データの中身の特徴（平均値や分散）などを勝手に計算して表示する機能や図としてプロットするようなこともしてくれます。  

また個人的に目玉の機能として、データベースのように検索するだけでなくデータの列に対して一括で任意の処理をすることができるのも魅力です。  

以下は日本大学の授業の一部を表形式にしたデータの例を示します。  

In [ ]:
import pandas as pd

classroom_info = pd.read_csv("https://admiralhonda-share-tech.on.drv.tw/python_ml_intro/data/class_select_app/class_info.csv")  # データの読み込み
classroom_info = classroom_info[["教科名","担当者","概要"]]
classroom_info.head(10)             # .head(数)で先頭何個かを取得できるぞ

In [ ]:
# @title 平均などの統計情報の取得
# .describe()でデータの平均とか中央値が見れます。
classroom_info.describe()                       # 文章の場合はデータの数などが表示される

In [ ]:
# @title 列操作

# 列の追加
classroom_info["test"] = 1
print(classroom_info.head())

"""
上のコードで新しく"test"という列ができたと思います
今回は要素をすべて1にしましたが、リストを渡すこともできます。
"""

# スーパー便利機能apply
"""
列の要素一つずつを対象とし、それを関数で処理して結果をリストにして新しく列を作成したり
更新することができます。
"""
def add(num : int) -> int:
    return num + 10

classroom_info["test"] = classroom_info["test"].apply(add)
print(classroom_info.head())

In [ ]:
# @title 課題

"""
classroom_info["概要"]の名詞の数を数える
数えた結果はclassroom["名詞の数"]に格納

ヒント１：
    入力の文章に含まれる名詞を数える関数を作成してapplyで適応させる
ヒント２：
    名詞を数える関数はmecabで形態素解析（分かち書き）し、指定項目に”名詞”と書いてあれば数える
"""

def culucute_meisi_num(sentense : str,m : MeCab.Tagger) -> int:
    """
    名詞の数を数える処理を記述しよう
    """
    pass

classroom_info["名詞の数"] = "ここに何か書いて上記の関数を適応してください"

print(classroom_info.head())